In [1]:
!pip install pysqlite3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pysqlite3: filename=pysqlite3-0.5.4-cp310-cp310-linux_x86_64.whl size=151321 sha256=0e32f4655a4dcd0456c6185e50358eb92b996a53e9c51125e7766eb0a0419043
  Stored in directory: /root/.cache/pip/wheels/04/55/74/e3b53aeb9ff9e933626ee3938ccad69123828115ae34ce028e
Successfully built pysqlite3


In [2]:
from pysqlite3 import dbapi2 as sqlite3
print(sqlite3.sqlite_version_info)

(3, 37, 2)


In [3]:
import pandas as pd
from IPython.display import display, HTML

In [4]:
dbname = "music_streaming_1.db"

def printSqlResults(cursor, tbl):
  df=pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
  display(HTML("<b><font color=Green>"+tbl+"</font></b>"+df.to_html(index=False)))

def runSql(caption, query):
  conn = sqlite3.connect(dbname)
  cursor=conn.cursor()
  cursor.execute(query)
  printSqlResults(cursor,caption)
  conn.close()

In [5]:
conn = sqlite3.connect(dbname)
cursor = conn.cursor()

cursor.execute("""
      CREATE TABLE IF NOT EXISTS Users (
        user_id INTEGER PRIMARY KEY,
        name VARCHAR(100) NOT NULL,
        email VARCHAR(100) NOT NULL UNIQUE
      );
""")

cursor.execute("""
  CREATE TABLE IF NOT EXISTS Songs (
    song_id INTEGER PRIMARY KEY,
    title VARCHAR(100) NOT NULL,
    artist VARCHAR(100) NOT NULL,
    genre VARCHAR(100)
  );
"""
)

cursor.execute("""
    CREATE TABLE IF NOT EXISTS Listens (
      listen_id INTEGER PRIMARY KEY,
      user_id INTEGER NOT NULL,
      song_id INTEGER NOT NULL,
      rating FLOAT,
      listen_time TIMESTAMP,
      FOREIGN KEY (user_id) REFERENCES Users(user_id),
      FOREIGN KEY (song_id) REFERENCES Songs(song_id)
    );
""")

cursor.execute("""
  CREATE TABLE IF NOT EXISTS Recommendations (
      recommendation_id INTEGER NOT NULL,
      recommendation_time TIMESTAMP,
      user_id INTEGER NOT NULL,
      song_id INTEGER NOT NULL,
      FOREIGN KEY (user_id) REFERENCES Users(user_id),
      FOREIGN KEY (song_id) REFERENCES Songs(song_id)
  );
""")

conn.commit()
conn.close()

In [6]:
conn = sqlite3.connect(dbname)
sqlite3.enable_callback_tracebacks(True)

cursor=conn.cursor()
cursor.execute("delete from Songs;")
cursor.execute("delete from Users;")
cursor.execute("delete from Listens;")
cursor.execute("delete from Recommendations;")

#Insert data into the table

cursor.execute("""
  INSERT INTO Users (user_id, name, email)
  VALUES
    (1, 'Mickey', 'mickey@example.com'),
    (2, 'Minnie', 'minnie@example.com'),
    (3, 'Daffy', 'daffy@example.com'),
    (4, 'Pluto', 'pluto@example.com');
""")
# Insert sample songs from Taylor Swift, Ed Sheeran, Beatles
cursor.execute("""
INSERT INTO Songs (song_id, title, artist, genre)
VALUES
    (1, 'Evermore', 'Taylor Swift', 'Pop'),
    (2, 'Willow', 'Taylor Swift', 'Pop'),
    (3, 'Shape of You', 'Ed Sheeran', 'Rock'),
    (4, 'Photograph', 'Ed Sheeran', 'Rock'),
    (5, 'Shivers', 'Ed Sheeran', 'Rock'),
    (6, 'Yesterday', 'Beatles', 'Classic'),
    (7, 'Yellow Submarine', 'Beatles', 'Classic'),
    (8, 'Hey Jude', 'Beatles', 'Classic'),
    (9, 'Bad Blood', 'Taylor Swift', 'Rock'),
    (10, 'DJ Mix', 'DJ', NULL);
""")

# Insert sample listens
cursor.execute("""
INSERT INTO Listens (listen_id, user_id, song_id, rating, listen_time)
VALUES
    (1, 1, 1, 4.5, '2024-08-30 14:35:00'),
    (2, 1, 2, 4.2, NULL),
    (3, 1, 6, 3.9, '2024-08-29 10:15:00'),
    (4, 2, 2, 4.7, NULL),
    (5, 2, 7, 4.6, '2024-08-28 09:20:00'),
    (6, 2, 8, 3.9, '2024-08-27 16:45:00'),
    (7, 3, 1, 2.9, NULL),
    (8, 3, 2, 4.9, '2024-08-26 12:30:00'),
    (9, 3, 6, NULL, NULL);
""")

# Commit changes and close the connection
conn.commit()
conn.close()

runSql('Users', "select * from Users;")
runSql('Songs', "select * from Songs;")
runSql('Listens', "select * from Listens;")

user_id,name,email
1,Mickey,mickey@example.com
2,Minnie,minnie@example.com
3,Daffy,daffy@example.com
4,Pluto,pluto@example.com


song_id,title,artist,genre
1,Evermore,Taylor Swift,Pop
2,Willow,Taylor Swift,Pop
3,Shape of You,Ed Sheeran,Rock
4,Photograph,Ed Sheeran,Rock
5,Shivers,Ed Sheeran,Rock
6,Yesterday,Beatles,Classic
7,Yellow Submarine,Beatles,Classic
8,Hey Jude,Beatles,Classic
9,Bad Blood,Taylor Swift,Rock
10,DJ Mix,DJ,None


listen_id,user_id,song_id,rating,listen_time
1,1,1,4.5,2024-08-30 14:35:00
2,1,2,4.2,None
3,1,6,3.9,2024-08-29 10:15:00
4,2,2,4.7,None
5,2,7,4.6,2024-08-28 09:20:00
6,2,8,3.9,2024-08-27 16:45:00
7,3,1,2.9,None
8,3,2,4.9,2024-08-26 12:30:00
9,3,6,NaN,None


In [7]:
qry_classic_songs = """
SELECT Songs.title, Songs.artist FROM Songs
WHERE Songs.genre= 'Classic';
"""
runSql('Classic Query', qry_classic_songs)

title,artist
Yesterday,Beatles
Yellow Submarine,Beatles
Hey Jude,Beatles


In [8]:
#Find titles and artists if songs in Classic Genre with titles starting like Ye...

qry_classic_songs2 = """
SELECT Songs.title, Songs.artist FROM Songs
WHERE Songs.genre= 'Classic' AND
  Songs.title LIKE 'Ye%';
"""
runSql('Classic Query with songs like Ye..', qry_classic_songs2)

title,artist
Yesterday,Beatles
Yellow Submarine,Beatles


In [9]:
#List all the genres in the Songs table
qry_genres = """
SELECT genre from Songs;
"""
runSql('All genres', qry_genres)

genre
Pop
Pop
Rock
Rock
Rock
Classic
Classic
Classic
Rock
None


In [10]:
#Find the unique genres from the Songs table

qry_distinct = """
SELECT DISTINCT genre
FROM Songs;
"""

runSql('Distinct Genres', qry_distinct)

genre
Pop
Rock
Classic
None


In [11]:
#Find the number of songs by Taylor Swift in different genres

qry_taylor_count="""
SELECT artist, genre,count(*) as num_songs
FROM Songs
WHERE Songs.artist='Taylor Swift'
GROUP BY artist, genre;
"""
runSql('Count of songs by Taylor Swift in different genres', qry_taylor_count)

artist,genre,num_songs
Taylor Swift,Pop,2
Taylor Swift,Rock,1


In [12]:
#Find the number of songs by all artists in different genres

qry_artist_count="""
SELECT artist, genre,count(*) as num_songs
FROM Songs
GROUP BY artist, genre;
"""
runSql('Count of songs by Taylor Swift in different genres', qry_artist_count)


artist,genre,num_songs
Beatles,Classic,3
DJ,None,1
Ed Sheeran,Rock,3
Taylor Swift,Pop,2
Taylor Swift,Rock,1


In [13]:
#Create one single table for songs and listens. Redundancies are allowed

qry_large_table = """
SELECT * FROM Songs
LEFT JOIN Listens ON Songs.song_id=Listens.song_id
LEFT JOIN Users ON Listens.user_id=Users.user_id;
"""
runSql('One Large Table',qry_large_table)

song_id,title,artist,genre,listen_id,user_id,song_id,rating,listen_time,user_id,name,email
1,Evermore,Taylor Swift,Pop,1.0,1.0,1.0,4.5,2024-08-30 14:35:00,1.0,Mickey,mickey@example.com
1,Evermore,Taylor Swift,Pop,7.0,3.0,1.0,2.9,None,3.0,Daffy,daffy@example.com
2,Willow,Taylor Swift,Pop,2.0,1.0,2.0,4.2,None,1.0,Mickey,mickey@example.com
2,Willow,Taylor Swift,Pop,4.0,2.0,2.0,4.7,None,2.0,Minnie,minnie@example.com
2,Willow,Taylor Swift,Pop,8.0,3.0,2.0,4.9,2024-08-26 12:30:00,3.0,Daffy,daffy@example.com
3,Shape of You,Ed Sheeran,Rock,NaN,NaN,NaN,NaN,None,NaN,None,None
4,Photograph,Ed Sheeran,Rock,NaN,NaN,NaN,NaN,None,NaN,None,None
5,Shivers,Ed Sheeran,Rock,NaN,NaN,NaN,NaN,None,NaN,None,None
6,Yesterday,Beatles,Classic,3.0,1.0,6.0,3.9,2024-08-29 10:15:00,1.0,Mickey,mickey@example.com
6,Yesterday,Beatles,Classic,9.0,3.0,6.0,NaN,None,3.0,Daffy,daffy@example.com


In [14]:
#Find the Highly Rates Songs. Consider Rating above 4.6 as highly rated

qry_rating_songs="""
SELECT Songs.song_id, Songs.title, Songs.artist, Listens.rating
FROM Songs
JOIN Listens ON Songs.song_id = Listens.song_id
WHERE Listens.rating > 4.6
"""
runSql('Highly Rated Songs',qry_rating_songs)

#Find tha average rating of every song

qry_avg_songs="""
SELECT Songs.song_id, Songs.title, Songs.artist, AVG(Listens.rating)
FROM Songs
JOIN Listens ON Songs.song_id = Listens.song_id
GROUP BY Songs.song_id, Songs.title, Songs.artist
"""
runSql('Highly Rated Songs',qry_avg_songs)



song_id,title,artist,rating
2,Willow,Taylor Swift,4.7
2,Willow,Taylor Swift,4.9


song_id,title,artist,AVG(Listens.rating)
1,Evermore,Taylor Swift,3.7
2,Willow,Taylor Swift,4.6
6,Yesterday,Beatles,3.9
7,Yellow Submarine,Beatles,4.6
8,Hey Jude,Beatles,3.9


In [15]:
#Find the popular songs by counting the listens

qry_pop_songs = """
SELECT Songs.song_id, Songs.title, Songs.artist, count(Listens.song_id)
FROM Songs
JOIN Listens
ON Songs.song_id=Listens.song_id
GROUP BY Songs.title, Songs.artist
ORDER BY COUNT(Listens.song_id) DESC;
"""

runSql('Popular Songs', qry_pop_songs)

song_id,title,artist,count(Listens.song_id)
2,Willow,Taylor Swift,3
1,Evermore,Taylor Swift,2
6,Yesterday,Beatles,2
8,Hey Jude,Beatles,1
7,Yellow Submarine,Beatles,1


In [16]:
#Find songs by Ed Sheeran and Taylor Swift

qry_edtaylor="""
SELECT title, artist
FROM Songs
WHERE artist IN ('Ed Sheeran', 'Taylor Swift');
"""
runSql('Songs by Ed Sheeran and Taylor Swift', qry_edtaylor)

#Find songs from both pop and rock genres

qry_popandrock = """
SELECT title, artist
FROM Songs
WHERE genre='Pop'
UNION
SELECT title, artist
FROM Songs
WHERE genre='Rock'
"""
runSql('Songs on both Pop and Rock', qry_popandrock)


title,artist
Evermore,Taylor Swift
Willow,Taylor Swift
Shape of You,Ed Sheeran
Photograph,Ed Sheeran
Shivers,Ed Sheeran
Bad Blood,Taylor Swift


title,artist
Bad Blood,Taylor Swift
Evermore,Taylor Swift
Photograph,Ed Sheeran
Shape of You,Ed Sheeran
Shivers,Ed Sheeran
Willow,Taylor Swift


In [17]:
#Find songs listened to by user_id 1

qry_listen_usr="""
SELECT title, artist
FROM Songs
WHERE song_id IN (SELECT song_id FROM Listens WHERE Listens.listen_time IS NULL);
"""
runSql('Songs listened by user_id=1',qry_listen_usr)

title,artist
Evermore,Taylor Swift
Willow,Taylor Swift
Yesterday,Beatles


In [18]:
#Computing the recommendations
#Find the song pairs which are shared across > 1 user

qry_sharing="""
WITH song_similarity AS (
SELECT u1.song_id as song1, u2.song_id as song2, COUNT(*) AS common_users
FROM LISTENS u1
JOIN LISTENS u2
ON u1.user_id=u2.user_id
AND u1.song_id != u2.song_id
GROUP BY u1.song_id, u2.song_id
HAVING COUNT(*)>1
),

recs AS (
  SELECT user_id, song2 as song_id
  FROM song_similarity
  JOIN Listens as L
  ON L.song_id = song_similarity.song1
  WHERE song_similarity.song2 NOT IN
  (SELECT song_id FROM Listens as temp where temp.user_id=L.user_id)
)
select * from recs;
"""
runSql('Songs shared', qry_sharing)

user_id,song_id
2,1
2,6


In [19]:
#ASSIGNMENT-2. To be submitted as a PDF with queries and screenshots.

#1 Insert the above into the recommendations table
#2 Generate the recommendaions for Minnie

#3 Re-do the generation of recommendations now on the basis of listen time
#4 Generate new recommendations
#5 What are the differences with the static method on #2 above

*Q1* #1 Insert the above into the recommendations table

In [23]:
import sqlite3
conn = sqlite3.connect(dbname)
cursor = conn.cursor()

recommendation_query_temp = """
INSERT INTO Recommendations (recommendation_id, recommendation_time, user_id,
song_id)
WITH song_similarity AS (
SELECT u1.song_id AS song1, u2.song_id AS song2, COUNT(*) AS common_users
FROM Listens u1
JOIN Listens u2
ON u1.user_id = u2.user_id AND u1.song_id != u2.song_id
GROUP BY u1.song_id, u2.song_id
HAVING COUNT(*) > 1
)
SELECT
ROW_NUMBER() OVER (ORDER BY L.user_id, song2) AS recommendation_id,
datetime('now') AS recommendation_time,
L.user_id,
song2 AS song_id
FROM song_similarity
JOIN Listens AS L ON L.song_id = song_similarity.song1
WHERE song2 NOT IN (
SELECT song_id FROM Listens WHERE user_id = L.user_id
);
"""
cursor.execute(recommendation_query_temp)
conn.commit()

runSql('TEST', 'SELECT * FROM Recommendations;')

recommendation_id,recommendation_time,user_id,song_id
1,2024-12-15 14:42:10,2,1
2,2024-12-15 14:42:10,2,6
1,2024-12-15 14:42:25,2,1
2,2024-12-15 14:42:25,2,6
1,2024-12-15 14:42:47,2,1
2,2024-12-15 14:42:47,2,6
1,2024-12-15 14:43:22,2,1
2,2024-12-15 14:43:22,2,6


#2 Generate the recommendaions for Minnie

In [25]:
minnie_recommendations_query = """
SELECT
s.title,
s.artist,
s.genre,
r.recommendation_time
FROM Recommendations r
JOIN Songs s ON r.song_id = s.song_id
WHERE r.user_id = 2
ORDER BY r.recommendation_time DESC;
"""
runSql('Minnie Recommendations', minnie_recommendations_query)

title,artist,genre,recommendation_time
Evermore,Taylor Swift,Pop,2024-12-15 14:43:22
Yesterday,Beatles,Classic,2024-12-15 14:43:22
Evermore,Taylor Swift,Pop,2024-12-15 14:42:47
Yesterday,Beatles,Classic,2024-12-15 14:42:47
Evermore,Taylor Swift,Pop,2024-12-15 14:42:25
Yesterday,Beatles,Classic,2024-12-15 14:42:25
Evermore,Taylor Swift,Pop,2024-12-15 14:42:10
Yesterday,Beatles,Classic,2024-12-15 14:42:10


#3 Re-do the generation of recommendations now on the basis of listen time

In [27]:

time_query = """
SELECT
l1.user_id,
s.title,
s.artist,
l1.rating,
l2.rating AS other_rating,
l1.listen_time,
l2.listen_time AS other_listen_time
FROM Listens l1
JOIN Listens l2 ON l1.user_id = l2.user_id AND l1.song_id != l2.song_id
JOIN Songs s ON l2.song_id = s.song_id
WHERE l1.listen_time IS NOT NULL
AND l2.listen_time IS NOT NULL
ORDER BY l1.user_id, l1.listen_time DESC;
"""

runSql('Time-based Analysis Results', time_query)

user_id,title,artist,rating,other_rating,listen_time,other_listen_time
1,Yesterday,Beatles,4.5,3.9,2024-08-30 14:35:00,2024-08-29 10:15:00
1,Evermore,Taylor Swift,3.9,4.5,2024-08-29 10:15:00,2024-08-30 14:35:00
2,Hey Jude,Beatles,4.6,3.9,2024-08-28 09:20:00,2024-08-27 16:45:00
2,Yellow Submarine,Beatles,3.9,4.6,2024-08-27 16:45:00,2024-08-28 09:20:00


#4 Generate new recommendations

In [35]:

cursor.execute("DELETE FROM Recommendations;")
conn.commit()
# ------ new recommendations based on user preferences ---
insert_new_recom_query = """
WITH user_preferences AS (
SELECT
l1.user_id,
l1.song_id,
s1.genre,
l1.rating,
l1.listen_time
FROM Listens l1
JOIN Songs s1 ON l1.song_id = s1.song_id
WHERE l1.listen_time IS NOT NULL
),
potential_recommendations AS (
SELECT
up.user_id,
s2.song_id AS recommended_song_id,
s2.title,
s2.artist,
s2.genre,
up.rating AS user_rating,
ROW_NUMBER() OVER (PARTITION BY up.user_id ORDER BY up.listen_time DESC,
up.rating DESC) AS rank
FROM user_preferences up
JOIN Songs s2 ON up.genre = s2.genre
WHERE s2.song_id NOT IN (
SELECT song_id FROM Listens WHERE user_id = up.user_id
)
)
INSERT INTO Recommendations (recommendation_id, recommendation_time, user_id,
song_id)
SELECT
ROW_NUMBER() OVER (ORDER BY user_id, rank),
datetime('now'),
user_id,
recommended_song_id
FROM potential_recommendations
WHERE rank <= 3;
"""


cursor.execute(insert_new_recom_query)
conn.commit()



verify_query = """
SELECT
r.user_id,
u.name AS user_name,
s.title,
s.artist,
s.genre,
r.recommendation_time
FROM Recommendations r
JOIN Songs s ON r.song_id = s.song_id
JOIN Users u ON r.user_id = u.user_id
ORDER BY r.user_id, r.recommendation_time;
"""
runSql('New Time-Based Recommendations Result', verify_query)

user_id,user_name,title,artist,genre,recommendation_time
1,Mickey,Hey Jude,Beatles,Classic,2024-12-15 15:05:29
1,Mickey,Yellow Submarine,Beatles,Classic,2024-12-15 15:05:29
2,Minnie,Yesterday,Beatles,Classic,2024-12-15 15:05:29
2,Minnie,Yesterday,Beatles,Classic,2024-12-15 15:05:29


#5 What are the differences with the static method on #2 above

In [37]:
static_analysis_query_test = """
SELECT
l.user_id,
u.name,
s.title AS listened_to,
s.artist,
l.rating,
l.listen_time
FROM Listens l
JOIN Songs s ON l.song_id = s.song_id
JOIN Users u ON l.user_id = u.user_id
ORDER BY l.user_id, l.rating DESC;
"""


runSql('Static Analysis Listening Patterns', static_analysis_query_test)

time_compr_query = """
SELECT
r.user_id,
u.name,
s.title AS recommended_song,
s.artist,
s.genre,
r.recommendation_time
FROM Recommendations r
JOIN Songs s ON r.song_id = s.song_id
JOIN Users u ON r.user_id = u.user_id
ORDER BY r.user_id, r.recommendation_time DESC;
"""

runSql('Time-Based Recommendations Resulkts : ', time_compr_query)

user_id,name,listened_to,artist,rating,listen_time
1,Mickey,Evermore,Taylor Swift,4.5,2024-08-30 14:35:00
1,Mickey,Willow,Taylor Swift,4.2,None
1,Mickey,Yesterday,Beatles,3.9,2024-08-29 10:15:00
2,Minnie,Willow,Taylor Swift,4.7,None
2,Minnie,Yellow Submarine,Beatles,4.6,2024-08-28 09:20:00
2,Minnie,Hey Jude,Beatles,3.9,2024-08-27 16:45:00
3,Daffy,Willow,Taylor Swift,4.9,2024-08-26 12:30:00
3,Daffy,Evermore,Taylor Swift,2.9,None
3,Daffy,Yesterday,Beatles,NaN,None


user_id,name,recommended_song,artist,genre,recommendation_time
1,Mickey,Hey Jude,Beatles,Classic,2024-12-15 15:05:29
1,Mickey,Yellow Submarine,Beatles,Classic,2024-12-15 15:05:29
2,Minnie,Yesterday,Beatles,Classic,2024-12-15 15:05:29
2,Minnie,Yesterday,Beatles,Classic,2024-12-15 15:05:29
